In [ ]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor

# Load the ONNX model
onnx_model = onnx.load("model.onnx")

input_name = "input.1" 
input_shape = {"input.1": (64, 3, 32, 32)}  # Update the input shape if necessary

# Convert the ONNX model to a Relay module
mod, params = relay.frontend.from_onnx(onnx_model, input_shape)

target = "cpu"  

# Build the model
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

# Create a TVM runtime executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

# Prepare input data (example: random input)
import numpy as np
input_data = np.random.uniform(size=input_shape[input_name]).astype("float32")
module.set_input(input_name, input_data)

# Run the model
module.run()

# Get the output
output = module.get_output(0)
print(output)


AssertionError: User specified the shape for inputs that weren't found in the graph: {'input.1': (64, 3, 32, 32)}

In [3]:
print(onnx_model.graph.input)


[name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 32
      }
    }
  }
}
]
